In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [23]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [24]:
def scrape_links(page_number):
    url=f"https://www.zonajakarta.com/indeks-berita?daterange=1%20August%202023%20-%2030%20September%202023&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h2',{"class": "latest__title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [25]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 1
Total Links: 10


In [26]:
results = []
def scrape_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        
        # Judul Berita
        title_elem = soup.find('h1',{"class": "read__title"})
        if title_elem:
            title_text = title_elem.text
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"class": "read__info__author"})
        if author_elem:
            author_text = author_elem.find('a')
            author_text = author_text.text
        else:
            author_text = "Author not found"     
        # tanggal berita
        date_elem = soup.find('div', {"class": "read__info__date"})
        if date_elem:
            date_text = date_elem.text.split('-')[1]
        else:
            date_text = "Date not found"
            
        #     # Category berita
        category_elements = soup.find_all('a', {"class": "breadcrumb__link"})
        if category_elements:
            category_text = category_elements[1].text
        else:
            category_text = "Category not found"
        #     # Content Berita
        body_elem = soup.find('article', {"class": "read__content clearfix"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'author' : author_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'region' : 'jakarta',
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [27]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [28]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 10


,title,author,category,date,content,region,link
0,"Lewat Maritime Training Acitivity Philindo, I...",Barri Zilhaq Vindia,Regional,"Rabu, 27 September 2023 | 09:46 WIB\n ...","\nZONAJAKARTA.com - Belum lama ini, Komandan G...",jakarta,https://www.zonajakarta.com/regional/673103042...
1,"Bentrokan Rempang Ditunggangi Pihak Asing, Me...",Barri Zilhaq Vindia,Nasional,"Rabu, 27 September 2023 | 06:05 WIB\n ...",\nZONAJAKARTA.com - Betrokan yang terjadi di P...,jakarta,https://www.zonajakarta.com/nasional/673103015...
2,"Tingkatkan Kekuatan Tempurnya, Menteri Pertah...",Barri Zilhaq Vindia,Internasional,"Rabu, 27 September 2023 | 10:12 WIB\n ...",\nZONAJAKARTA.com - Siapa sangka Rusia sudah m...,jakarta,https://www.zonajakarta.com/internasional/6731...
3,Media Asing Sebut F-16 Block 15 Indonesia Kun...,Beryl Santoso,Nasional,"Rabu, 27 September 2023 | 06:25 WIB\n ...",\nZONAJAKARTA.com - Saat ini Indonesia tengah ...,jakarta,https://www.zonajakarta.com/nasional/673103024...
4,"F-15 Indonesia Sanggup Operasional 50 Tahun, ...",Beryl Santoso,Nasional,"Rabu, 27 September 2023 | 06:08 WIB\n ...",\nZONAJAKARTA.com - F-15 Eagle II menjadi sala...,jakarta,https://www.zonajakarta.com/nasional/673103023...
5,AS Secara Tidak Langsung Bantu Rusia Hancurka...,Barri Zilhaq Vindia,Internasional,"Rabu, 27 September 2023 | 06:10 WIB\n ...","\nZONAJAKARTA.com - Sudah bisa dipastikan, pih...",jakarta,https://www.zonajakarta.com/internasional/6731...
6,Indonesia Harus Sadar Penting Miliki Sebuah D...,Barri Zilhaq Vindia,Teknologi,"Rabu, 27 September 2023 | 06:14 WIB\n ...",\nZONAJAKARTA.com - Platform tak berawak atau ...,jakarta,https://www.zonajakarta.com/teknologi/67310302...
7,Pengamat Pertahanan Peringatkan Sukhoi Rusia ...,Beryl Santoso,Nasional,"Rabu, 27 September 2023 | 05:31 WIB\n ...",\nZONAJAKARTA.com - Indonesia sejatinya mengin...,jakarta,https://www.zonajakarta.com/nasional/673103021...
8,F-35 Australia dan F-16 Indonesia Sama-sama P...,Zulaika Rizkia,Nasional,"Selasa, 26 September 2023 | 21:43 WIB\n ...",\nZONAJAKARTA.COM- Pertama kali diadakan pada ...,jakarta,https://www.zonajakarta.com/nasional/673103006...
9,AS Kirim Tank Abrams ke Ukraina untuk Hadapi ...,Barri Zilhaq Vindia,Internasional,"Rabu, 27 September 2023 | 09:15 WIB\n ...",\nZONAJAKARTA.com - Salah satu pasokan senjata...,jakarta,https://www.zonajakarta.com/internasional/6731...


In [29]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/zonejakarta_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')